##PEFT Training
Traditional finetuning
###Imports

In [ ]:
!pip install -U transformers datasets scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, get_scheduler
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import numpy as np
from datasets import load_dataset

###Make 10% of the training data for validation

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
ds = load_dataset("Salesforce/cos_e", "v1.11", split={"train": "train", "test": "validation"})

def modify_example(example):
    choice_labels = ['(a)', '(b)', '(c)', '(d)', '(e)']
    formatted_choices = "\n".join([f"{choice_labels[i]} {choice}" for i, choice in enumerate(example["choices"])])
    input_text = f"{example['question']}\nAnswer Choices:\n{formatted_choices}"
    return {
        "input": input_text,
        "label": example["answer"]
    }

dataset = {split: data.map(modify_example, remove_columns=['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation']) for split, data in ds.items()}

# Create validation split from original training data
val_dataset = dataset["train"]
val_size = int(0.1 * len(val_dataset))  # 10% for validation
train_size = len(val_dataset) - val_size

# Create the split
train_set, val_set = random_split(
    val_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/222k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [ ]:
data_splits = {'train': train_set, 'val': val_set, 'test': dataset['test']}

for split_name, split_data in data_splits.items():
    print(f"Length of {split_name}_set:", len(split_data))
    print(f"First element of {split_name}_set:", split_data[0])

Length of train_set: 8767
First element of train_set: {'input': 'Where might someone keep personal soap?\nAnswer Choices:\n(a) birthday party\n(b) supermarket\n(c) own home\n(d) jail\n(e) cabinet', 'label': 'own home'}
Length of val_set: 974
First element of val_set: {'input': 'What do you have to do to learn to play violin?\nAnswer Choices:\n(a) tune\n(b) practise\n(c) relaxing\n(d) ask questions\n(e) take lessons', 'label': 'take lessons'}
Length of test_set: 1221
First element of test_set: {'input': 'A beaver is know for building prowess, their supplies come from where?\nAnswer Choices:\n(a) british columbia\n(b) body of water\n(c) wooded area\n(d) pay debts\n(e) zoo', 'label': 'wooded area'}


In [ ]:
class T5FineTuningDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]

        # Tokenize inputs
        input_encoding = self.tokenizer(
            example['input'],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Tokenize labels
        label_encoding = self.tokenizer(
            example['label'],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Replace padding token ID with -100
        labels = label_encoding.input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_encoding.input_ids.squeeze(),
            'attention_mask': input_encoding.attention_mask.squeeze(),
            'labels': labels.squeeze()
        }

In [ ]:
def validate_model(model, val_loader, device):
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    return total_val_loss / len(val_loader)

###Training

In [ ]:
def train_model(train_set, val_set, model_name="google-t5/t5-small", max_length=1024,
                batch_size=2, num_epochs=25, learning_rate=3e-5, weight_decay=0.01):  # reduced batch size from 16 to 2 due to CUDA memory error

    # Load model and tokenizer
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    device = "cuda"
    model.to(device)

    # Prepare training and validation datasets
    train_dataset = T5FineTuningDataset(train_set, tokenizer, max_length)
    val_dataset = T5FineTuningDataset(val_set, tokenizer, max_length)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    total_steps = len(train_loader) * num_epochs

    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Initialize variables to track best model
    best_val_loss = float('inf')
    best_model_state = None

    # Training loop
    print("Starting training...")
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Training
        model.train()
        total_train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}")

        for batch in progress_bar:
            # Move batch to device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            total_train_loss += loss.item()

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()

            progress_bar.set_postfix({"loss": loss.item()})

        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch+1} - Average training loss: {avg_train_loss:.4f}")

        # Validation
        print("Running validation...")
        val_loss = validate_model(model, val_loader, device)
        print(f"Epoch {epoch+1} - Validation loss: {val_loss:.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            print(f"New best model saved with validation loss: {val_loss:.4f}")

    # Load best model for final evaluation
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Loaded best model with validation loss: {best_val_loss:.4f}")

    print("Training completed!")
    return model, tokenizer

In [ ]:
model, tokenizer = train_model(train_set, val_set)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

...

Epoch 23 - Average training loss: 0.2273
Running validation...
Epoch 23 - Validation loss: 1.0233
Epoch 24/25


Training Epoch 24:   0%|          | 0/4384 [00:00<?, ?it/s]

Epoch 24 - Average training loss: 0.2299
Running validation...
Epoch 24 - Validation loss: 1.0273
Epoch 25/25


Training Epoch 25:   0%|          | 0/4384 [00:00<?, ?it/s]

Epoch 25 - Average training loss: 0.2168
Running validation...
Epoch 25 - Validation loss: 1.0284
Loaded best model with validation loss: 0.6685
Training completed!


###Evaluation

In [ ]:
def evaluate_model(model, tokenizer, test_dataset, max_length=1024, batch_size=16):
    device = "cuda"
    model.to(device)

    # Prepare test dataset
    test_dataset_processed = T5FineTuningDataset(test_dataset, tokenizer, max_length)
    test_loader = DataLoader(test_dataset_processed, batch_size=batch_size)

    model.eval()
    predictions = []
    references = []

    print("Starting evaluation...")
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Generate predictions
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length
            )

            # Decode predictions and references
            preds = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

            # Replace -100 with pad token id first
            label_ids = labels.clone()
            label_ids[label_ids == -100] = tokenizer.pad_token_id
            refs = [tokenizer.decode(label, skip_special_tokens=True) for label in label_ids]

            predictions.extend(preds)
            references.extend(refs)

    # Calculate accuracy
    accuracy = accuracy_score(references, predictions) * 100
    return accuracy, predictions, references

In [ ]:
accuracy, predictions, references = evaluate_model(model, tokenizer, dataset["test"])

Starting evaluation...


Evaluating:   0%|          | 0/77 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_pred = type_of_target(y_pred, input_name="y_pred")


###Results

In [ ]:
# 2nd run - Final Evaluation Accuracy: %
def display_evaluation_results(accuracy, predictions, references, num_samples=5):

    print(f"Final Evaluation Accuracy: {accuracy:.2f}%")

    # Display some examples
    print("\nSample predictions:")
    for i in range(min(num_samples, len(predictions))):
        print(f"Reference: {references[i]}")
        print(f"Prediction: {predictions[i]}")
        print("-" * 50)


In [ ]:
display_evaluation_results(accuracy, predictions, references, num_samples=5)

Final Evaluation Accuracy: 43.98%

Sample predictions:
Reference: wooded area
Prediction: wooded area
--------------------------------------------------
Reference: go downtown
Prediction: go downtown
--------------------------------------------------
Reference: play tag
Prediction: play tag
--------------------------------------------------
Reference: great outdoors
Prediction: corn fields
--------------------------------------------------
Reference: club
Prediction: meeting
--------------------------------------------------


In [ ]:
# 1st run - Final Evaluation Accuracy: 42.67%
def display_evaluation_results(accuracy, predictions, references, num_samples=5):

    print(f"Final Evaluation Accuracy: {accuracy:.2f}%")

    # Display some examples
    print("\nSample predictions:")
    for i in range(min(num_samples, len(predictions))):
        print(f"Reference: {references[i]}")
        print(f"Prediction: {predictions[i]}")
        print("-" * 50)


In [ ]:
display_evaluation_results(accuracy, predictions, references, num_samples=5)

Final Evaluation Accuracy: 42.67%

Sample predictions:
Reference: wooded area
Prediction: british columbia
--------------------------------------------------
Reference: go downtown
Prediction: go downtown
--------------------------------------------------
Reference: play tag
Prediction: play tag
--------------------------------------------------
Reference: great outdoors
Prediction: corn fields
--------------------------------------------------
Reference: club
Prediction: meeting
--------------------------------------------------


Results vary between runs.